In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sympy import symbols, diff

import warnings 
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format ='retina'
sns.set(style='ticks')

plt.rc('figure', figsize=(6, 3.7), dpi=100) # figure size
plt.rc('axes', labelpad=20, facecolor="#ffffff", # properties of axes
       linewidth=0.4, grid=True, labelsize=10) 
plt.rc('xtick.major', width=0.2) # major ticks where the labels are shown
plt.rc('ytick.major', width=0.2) # minor ticks where the labels are not shown
plt.rc('grid', color='#EEEEEE', linewidth=0.25)
plt.rc('font', family='Arial', weight='400', size=10)
plt.rc('text', color='#282828')
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('savefig', pad_inches=0.3, dpi=300)

In [ ]:
tube_dia_m = 62.3E-03
roughness = 30.0E-06
area_cross_section = (math.pi*(tube_dia_m**2))/4
depth = 0
depth_total = 3000
Pwh = 9.393E06
Qgsc = 8.0
velocity_surface = Qgsc/area_cross_section
gas_density_standard = 0.95
gas_gravity = .77
FBHT = 120
FTHT = 30
temp_gradient = (FBHT-FTHT)/depth_total
well_inclination = math.pi/2
dimensionless_roughness = roughness/tube_dia_m
molecular_weight = 28.97*gas_gravity
T = FBHT


In [ ]:
def newton_raphson(f, x_symbolic, x0, max_iter, tol=1e-06 ):
    x = x0
    f_prime = diff(f, x_symbolic)
    
    for i in range(max_iter):
        f_prime_val  = f_prime.subs(x_symbolic, x)
        x_new = x- f.subs(x_symbolic, x)/f_prime_val
        
        if abs(x_new - x)< tol:
            return x_new
        
        x = x_new
        
    raise ValueError('cannot converge')
        
    

In [ ]:
nstep = 301
loss_data = np.zeros((300, 6))

delta_depth = 10
depth = 0
sum_delta_depth = 0
P = Pwh
P_head_loss = Pwh
P_fric_loss = Pwh
P_acc_loss = Pwh
    
for i in range(nstep):
    depth_m = depth+sum_delta_depth
    T = temp_gradient*depth_m + FTHT
    T_ran = (T*9/5)+ 491.67
    T_abs = T+273.15
    P_pc_psi = 756.8 -131.07*gas_gravity - 3.6*gas_gravity**2
    P_pc_pa = P_pc_psi*6894.76
    T_pc_ran = 169.2+349.5*gas_gravity - 74*gas_gravity**2
    T_pc_k = T_pc_ran*5/9
    t = T_pc_ran/T_ran
    A = 0.06125*t*math.exp(-1.2*(1-t)**2)/P_pc_psi

    x = symbols('x')
    f = (-A*P/6894.76)+((x+(x**2)+(x**3)-x**4)/(1-x)**3)-((14.76*t-(9.76*t**2)+4.58*t**3)*x**2) +(90.7*t - (242*t**2)+42.4*t**3)*x**(2.18+2.82*t)

    x0= 0.2
    x2 = newton_raphson(f, x, x0, 1000)

    Z= A*P/(6894.76*x2)


    # computation of dzdp
    h = 1e-6
    P_h = P+h

    y = symbols('y')
    l = (-A*P_h/6894.76)+((y+(y**2)+(y**3)-y**4)/(1-y)**3)-((14.76*t-(9.76*t**2)+4.58*t**3)*y**2)+(90.7*t - 242*t**2+42.4*t**3)*y**(2.18+2.82*t)

    y0 = 0.2
    y2 = newton_raphson(l, y, y0, 1000)

    Z_h = A*P_h/(6894.76*y2)
    dzdp = (Z_h - Z)/(P_h - P)

    #  computation of density and viscosity

    density = 0.00149406*P*molecular_weight/(Z*T_ran*6894.76)
    density_si = density*1000
    K1 = (0.00094+0.000002*molecular_weight)*(T_ran**1.5)/(200+19*molecular_weight+T_ran)
    X = 3.5+(986/T_ran)+(0.01*molecular_weight)
    Y = 2.4 - 0.2*X
    viscosity = K1*math.exp(X*(density**Y))
    viscosity_pas = viscosity*0.001
    Bg = gas_density_standard/density_si
    velocity = Bg*(Qgsc+1)/area_cross_section
    Nre = 4*density_si*(Qgsc+1)/(np.pi * viscosity_pas*tube_dia_m)
    ff = 0.25/(math.log10((dimensionless_roughness/3.7)+(5.74/Nre**0.9)))**2
    coefficient_dpds_acc = -(density_si*(velocity**2)*((1/P)-(dzdp/Z)))
    dpds_grav = density_si*9.8*math.sin(well_inclination)
    head_loss = dpds_grav*delta_depth
    P_head_loss = P_head_loss + head_loss
    dpds_fric = (density_si*velocity**2*ff/(2*tube_dia_m))
    friction_loss = dpds_fric*delta_depth
    P_fric_loss = P_fric_loss + friction_loss
    dpds_acc = (dpds_grav+dpds_fric)*coefficient_dpds_acc/(1-coefficient_dpds_acc)
    acceleration_loss = dpds_acc*delta_depth
    P_acc_loss = P_acc_loss + acceleration_loss
    total_loss = head_loss + friction_loss + acceleration_loss
    sum_delta_depth = sum_delta_depth + delta_depth
    P = P+total_loss
    
    loss_data[i, 1]= P_head_loss
    loss_data[i, 2]= P_acc_loss
    loss_data[i, 3]= P_fric_loss
    loss_data[i, 4]= P
    loss_data[i, 5]= depth_m



In [ ]:
plt.plot(loss_data[:, 4], Depth_Data, label='Total_Loss')
plt.plot(Head_Loss_Data, Depth_Data, label='Head_Loss')
plt.plot(Fric_Loss_Data, Depth_Data, label='Fric_Loss')
plt.plot(Acc_Loss_Data, Depth_Data, label='Acc_Loss')
plt.xlabel('Pressure')
plt.ylabel('Depth')
plt.title('Pressure Traverse Curve')
plt.legend()
plt.gca().invert_yaxis()
plt.show()

In [ ]:
tube_dia_m = 62.3E-03
roughness = 30.0E-06
area_cross_section = (math.pi*(tube_dia_m**2))/4
depth = 0
depth_total = 3000
Pwf = 2.9E07
Qgsc = 8.0
velocity_surface = Qgsc/area_cross_section
gas_density_standard = 0.95
gas_gravity = .77
FBHT = 120
FTHT = 30
temp_gradient = (FBHT-FTHT)/depth_total
well_inclination = math.pi/2
dimensionless_roughness = roughness/tube_dia_m
molecular_weight = 28.97*gas_gravity
T = FBHT

In [ ]:
nstep = 301
result = np.zeros((10, 3))


for Qgsc in range(6):
    delta_depth = 10
    sum_delta_depth = 0
    P = Pwf
    depth = 3000
    P_head_loss = Pwf
    P_fric_loss = Pwf
    P_acc_loss = Pwf
    
    for i in range(nstep):
        depth_m = depth-sum_delta_depth
        T = temp_gradient*depth_m + FTHT
        T_ran = (T*9/5)+ 491.67
        T_abs = T+273.15
        P_pc_psi = 756.8 -131.07*gas_gravity - 3.6*gas_gravity**2
        P_pc_pa = P_pc_psi*6894.76
        T_pc_ran = 169.2+349.5*gas_gravity - 74*gas_gravity**2
        T_pc_k = T_pc_ran*5/9
        t = T_pc_ran/T_ran
        A = 0.06125*t*math.exp(-1.2*(1-t)**2)/P_pc_psi

        x = symbols('x')
        f = (-A*P/6894.76)+((x+x**2+x**3-x**4)/(1-x)**3)-((14.76*t-9.76*t**2+4.58*t**3)*x**2) +(90.7*t - 242*t**2+42.4*t**3)*x**(2.18+2.82*t)

        x0= 0.2
        x2 = newton_raphson(f, x, x0, 100)

        Z= A*P/(6894.76*x2)


        # computation of dzdp
        h = 1e-6
        P_h = P+h
    
        y = symbols('y')
        l = (-A*P_h/6894.76)+((y+y**2+y**3-y**4)/(1-y)**3)-((14.76*t-9.76*t**2+4.58*t**3)*y**2)+(90.7*t - 242*t**2+42.4*t**3)*y**(2.18+2.82*t)

        y0 = 0.2
        y2 = newton_raphson(l, y, y0, 100)

        Z_h = A*P_h/(6894.76*y2)
        dzdp = (Z_h - Z)/(P_h - P)

        #  computation of density and viscosity

        density = 0.00149406*P*molecular_weight/(Z*T_ran*6894.76)
        density_si = density*1000
        K1 = (0.00094+0.000002*molecular_weight)*(T_ran**1.5)/(200+19*molecular_weight+T_ran)
        X = 3.5+(986/T_ran)+(0.01*molecular_weight)
        Y = 2.4 - 0.2*X
        viscosity = K1*math.exp(X*(density**Y))
        viscosity_pas = viscosity*0.001
        Bg = gas_density_standard/density_si
        velocity = Bg*(Qgsc+1)/area_cross_section
        Nre = 4*density_si*(Qgsc+1)/(np.pi * viscosity_pas*tube_dia_m)
        ff = 0.25/(math.log10((dimensionless_roughness/3.7)+(5.74/Nre**0.9)))**2
        coefficient_dpds_acc = -(density_si*(velocity**2)*((1/P)-(dzdp/Z)))
        dpds_grav = density_si*9.8*math.sin(well_inclination)
        head_loss = dpds_grav*delta_depth
        P_head_loss = P_head_loss - head_loss
        dpds_fric = (density_si*velocity**2*ff/(2*tube_dia_m))
        friction_loss = dpds_fric*delta_depth
        P_fric_loss = P_fric_loss - friction_loss
        dpds_acc = (dpds_grav+dpds_fric)*coefficient_dpds_acc/(1-coefficient_dpds_acc)
        acceleration_loss = dpds_acc*delta_depth
        P_acc_loss = P_acc_loss - acceleration_loss
        total_loss = head_loss + friction_loss + acceleration_loss
        sum_delta_depth = sum_delta_depth - delta_depth
        P = P-total_loss
    

    result[Qgsc, 1]= P
    result[Qgsc, 2] = Qgsc+1


In [ ]:
plt.plot(Pwh_Data, Depth_Data, label='Total_Loss')
plt.xlabel('Pressure')
plt.ylabel('Depth')
plt.title('Pressure Traverse Curve')
plt.legend()
plt.gca().invert_yaxis()
plt.show()